In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as Layers
import pandas as pd
import numpy as np
import librosa
import os
from librosa.display import specshow as ss
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split as tts
!pip install noisereduce

In [ ]:
#If used with TPU

#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
#load the data into a dataset for further mapping
with open('../input/birdclef-2022/scored_birds.json','r') as sb:
    scored_birds = json.load(sb)
df = pd.read_csv('/kaggle/input/birdclef-2022/train_metadata.csv')
df = df[['primary_label', 'filename']]
labels = df['primary_label']
filename = df['filename']

In [ ]:
def return_filepath(filename):
    main = '../input/birdclef-2022/train_audio'
    return main + '/' + filename
#with tpu_strategy.scope():
#the spectrogram preprocessing function
def load_spectrogram(filepath):
    le = 32000*5
    mel,sr = librosa.load(filepath, sr = 32000)
    mel = ((mel-np.amin(mel))*2)/(np.amax(mel)-np.amin(mel)) - 1
    org_len = len(mel)
    intervals = librosa.effects.split(mel, top_db= 15, ref= np.max)
    intervals = intervals.tolist()
    mel = (mel.flatten()).tolist()
    nonsilent_y = []
    for p,q in intervals :
        nonsilent_y = nonsilent_y + mel[p:q+1] 
    mel = np.array(nonsilent_y).astype('float32')
    print(type(mel))
    if len(mel) < le:
        while len(mel) < le:
            mel = np.concatenate((mel,mel))
    mel = mel[:le]

    mel = np.abs(librosa.stft(mel,n_fft = 512, hop_length=256))
    mel = librosa.amplitude_to_db(mel, ref=np.max)
    mel = librosa.feature.melspectrogram(mel,fmin = 1000,fmax = 8000, sr=32000, n_fft=2048, hop_length=1024)
    mel = librosa.power_to_db(mel, ref=np.max)
    mel = np.squeeze(mel, axis = -1)
    return mel

'''
# Warp the linear scale spectrograms into the mel-scale.
    num_spectrogram_bins = stfts.shape[-1]
    lower_edge_hertz, upper_edge_hertz, num_mel_bins = 70.0, 8000.0, 2048
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(num_mel_bins, num_spectrogram_bins, 
                                                                        sr, lower_edge_hertz,
                                                                        upper_edge_hertz)
    mel_spectrograms = tf.tensordot(spectrograms, linear_to_mel_weight_matrix, 1)
    mel_spectrograms.set_shape(spectrograms.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))

# Compute a stabilized log to get log-magnitude mel-scale spectrograms.
    log_mel_spectrograms = tf.math.log(mel_spectrograms + 1e-6)

# Compute MFCCs from log_mel_spectrograms and take the first 13.
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrograms)[...,1024]
    mfcss = tf.expand_dims(mfcss, )
    mfcss = np.array(mfccs)'''
    

In [ ]:
le = 160000
mel,mel1 = load_spectrogram('../input/birdclef-2022/train_audio/bubsan/XC591398.ogg')
print(mel.shape)
librosa.display.specshow(mel, y_axis='mel', fmax=8000, x_axis='time');
plt.title('Mel Spectrogram');
plt.colorbar(format='%+2.0f dB');

from IPython.display import Audio as a

a(mel1,rate = 32000)

In [ ]:
mel.shape

In [ ]:
final_df = []
for x,y in df.iterrows():
    label, path = y
    if label in scored_birds:
        final_df.append([label,path])
    else:
        pass
final_df = pd.DataFrame(final_df)
final_df[1] = final_df[1].apply(return_filepath)


In [ ]:
# one_hot_encoding_for_bird_vectors
bird_dict = {}
for i in (scored_birds):
    y = scored_birds.index(i)
    bird_dict[i] = y
    
def changer(x):
    return bird_dict[x]
    

In [ ]:
#check check hehehehe
tester1 = final_df[0]
bird_list = tester1.unique()

In [ ]:
#ehem....checking again hehe (cuz i ain't sure ever)
tester = pd.get_dummies(tester1).values


In [ ]:
del(specs)
specs

In [ ]:
specs = []
#with tpu_strategy.scope():
for i in final_df[1]:
    x = load_spectrogram(i)
    specs.append(x)
specs = np.array(specs)

In [ ]:
print(specs[7].shape)
print(specs.shape)
tester.shape

In [ ]:
xtrain,xtest,ytrain,ytest = tts(specs, tester, train_size = 0.9, random_state=30)

In [ ]:
xtrain,xval,ytrain,yval = tts(xtrain,ytrain,train_size = 0.8, random_state = 30)

In [ ]:
del(specs)

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, baseline=0.6, patience =20)

In [ ]:
#with tpu_strategy.scope():
def make_model(shape,num_classes):
    input_ = Layers.Input((shape))
    x = Layers.Conv1D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(input_)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.Conv2D((1),(3,3), activation = 'relu',  padding = 'same',input_shape = shape)(x)
    x = Layers.MaxPooling2D(3,1)(x)
    x = Layers.Flatten()(x)
    x = Layers.Dense(128, activation = 'relu')(x)
    x = Layers.Dropout(0.3)(x)
    x = Layers.Dense(128, activation = 'relu')(x)
    x = Layers.Dropout(0.3)(x)
    x = Layers.Dense(128, activation = 'relu')(x)
    x = Layers.Dropout(0.3)(x)
    x = Layers.Dense(128, activation = 'relu')(x)
    x = Layers.Dropout(0.3)(x)
    x = Layers.Dense(128, activation = 'relu')(x)
    x = Layers.Dropout(0.3)(x)
    x = Layers.Dense(128, activation = 'relu')(x)
    x = Layers.Dropout(0.2)(x)
    output = Layers.Dense(num_classes, activation = 'softmax')(x)
    model = tf.keras.Model(input_, output)
    return model
model = make_model(xtrain.shape[1:], 21)
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy',
                                                                        'mae','Recall'])

import gc


In [ ]:
del(model)
gc.collect()

In [ ]:
model.summary()

In [ ]:
model.fit(xtrain,ytrain,batch_size = 2, epochs = 50,validation_data = (xval,yval),callbacks = [es],verbose = 1)

In [ ]:
x =model.evaluate(xtest,ytest)

In [ ]:
x = model.predict(np.array([xtest[4]]))
y = list(ytest[35])
print(x,y)

In [ ]:
#some soft preprocessing on the predicted output data hehe :p
def exact(x):
    for i in range(len(x)):
        if x[i]<0.2:
            x[i] = 0
        else:
            continue
    return x
#some writing for wav file detection hehe :p
def predictor(x):
    padders = tf.zeros([32000*5] - tf.shape(x), dtype = tf.float32)
    x = tf.concat([padders, x], 0)
    x = librosa.stft(np.array(x))
    x = tf.abs(x)
    x = tf.transpose(x)
    x = tf.keras.utils.normalize(x, order=2)
    x = tf.expand_dims(x,axis = -1)
    x = np.array([x])
    pred = exact(list(model.predict(np.array([xtest[1]]))[0]))
    return pred

In [ ]:
x,sr = librosa.load('../input/birdclef-2022/test_soundscapes/soundscape_453028782.ogg', sr = 16000)
test_file = x[176000:176000+80000]
birds = predictor(test_file)
print(birds)

In [ ]:
file_main = '../input/birdclef-2022/test_soundscapes'
test_csv = pd.read_csv('../input/birdclef-2022/test.csv')
def return_files(dirpath):
    finres = {'row_id':[], 'target':[]}
    for filename in os.listdir(dirpath):
        filepath = os.path.join(dirpath,filename)
        start = 0
        x,sr = librosa.load(filepath,sr = 32000)
        while start<x.shape[0]:
            end = start+32000*5
            print(f'end  = {end}')
            test_sample = x[start:end]
            pred_list = predictor(test_sample)
            for i in range(len(scored_birds)):
                temp_filename = filename[:len(filename)-4]+'_'+scored_birds[i]+'_'+str(end//32000)
                if pred_list[i]!=0:
                    finres['row_id'].append(temp_filename)
                    finres['target'].append(True)
                else:
                    finres['row_id'].append(temp_filename)
                    finres['target'].append(False)
            start = start+80000
    return finres

In [ ]:
x = pd.DataFrame(return_files('../input/birdclef-2022/test_soundscapes'))

In [ ]:
x.to_csv('submission.csv',index = False)